### `@author:` `Anmol Parida`

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. <br>Please import the following libraries to get started.

## Problem Statement

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.


The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:
- Thumbs up:  Increase the volume
- Thumbs down: Decrease the volume
- Left swipe: 'Jump' backwards 10 seconds
- Right swipe: 'Jump' forward 10 seconds  
- Stop: Pause the movie
 
Each video is a sequence of 30 frames (or images).

The data is in a zip file. The zip file contains a 'train' and a 'val' folder with two CSV files for the two folders. These folders are in turn divided into subfolders where each subfolder represents a video of a particular gesture. Each subfolder, i.e. a video, contains 30 frames (or images). Note that all images in a particular video subfolder have the same dimensions but different videos may have different dimensions. Specifically, videos have two types of dimensions - either 360x360 or 120x160 (depending on the webcam used to record the videos). Hence, you will need to do some pre-processing to standardise the videos. 
 
Each row of the CSV file represents one video and contains three main pieces of information - the name of the subfolder containing the 30 images of the video, the name of the gesture and the numeric label (between 0-4) of the video.
 
Your task is to train a model on the 'train' folder which performs well on the 'val' folder as well (as usually done in ML projects). We have withheld the test folder for evaluation purposes - your final model's performance will be tested on the 'test' set.
 
To get started with the model building process, you first need to get the data on your storage. 
In order to get the data on the storage, perform the following steps in order
- Open the terminal
- go down https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
- unzip Project_data.zip
 

# Solution

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import imageio

In [3]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
# Pointing to the Current Path
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10

In [6]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
# batch_size = 20

# <a id='generator'>Generator</a>
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

**Going forward we use these variables**<br>
- `x : number of frames`
- `y : image width`
- `z : image height`

In [7]:
x = 30
y = 120
z = 120

In [8]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'

num_train_sequences = len(train_doc)
num_val_sequences = len(val_doc)
num_epochs = 10

print('validation sequences =', num_val_sequences)
print('training sequences =', num_train_sequences)
print ('epochs =', num_epochs)

validation sequences = 100
training sequences = 663
epochs = 10


## Model
Here you make the model using different functionalities that Keras provides.<br> Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model.<br> You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. <br>Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

### `Experiment 01`

### Model_1

In [11]:
model_1 = Sequential()

model_1.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_1.add(Flatten())

model_1.add(Dense(1000, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(500, activation='relu'))
model_1.add(Dropout(0.5))

#Softmax Layer
model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = optimizers.Adam(lr=0.001)
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

**Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.**

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

**The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.**

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [16]:
model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10Epoch 1/10

67/67 [==============================] - 81s 1s/step - loss: 5.7407 - categorical_accuracy: 0.3159 - val_loss: 5.7588 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-5.78767-0.31222-5.75878-0.28000.h5
Epoch 2/10
67/67 [==============================] - 25s 369ms/step - loss: 3.9344 - categorical_accuracy: 0.2985 - val_loss: 4.4725 - val_categorical_accuracy: 0.2600

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-3.93438-0.29851-4.47254-0.26000.h5
Epoch 3/10
67/67 [==============================] - 25s 381ms/step - loss: 2.1746 - categorical_accuracy: 0.2935 - val_loss: 1.3400 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-2.17455-0.29353-1.33995-0.44000.h5
Epoch 4/10
67/67 [==============================] - 26s 38

`categorical_accuracy`  = 44% and `val_categorical_accuracy` = 58%, we will experment further.

In [17]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3 # RBG

x = 30 # number of frames
y = 120 # image width
z = 120 # image height

In [18]:
def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

### `Experiment 02`

# <a id='model_2'>Model_2</a>

In [19]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [20]:
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(classes, activation='softmax'))

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [21]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [22]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 127s 2s/step - loss: 1.7772 - categorical_accuracy: 0.1985 - val_loss: 1.6094 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-1.77902-0.20060-1.60937-0.21000.h5
Epoch 2/10
67/67 [==============================] - 45s 667ms/step - loss: 1.6089 - categorical_accuracy: 0.2189 - val_loss: 1.6086 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-1.60886-0.21891-1.60857-0.21000.h5
Epoch 3/10
67/67 [==============================] - 45s 667ms/step - loss: 1.6133 - categorical_accuracy: 0.1393 - val_loss: 1.6079 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.61326-0.13930-1.60788-0.21000.h5
Epoch 4/10
67/67 [==============================] - 45s 6

Both `categorical_accuracy` and `val_categorical_accuracy` are at max 21%. Deleting all h5 files.

### `Experiment 03`

In [23]:
x = 30 # number of frames
y = 60 # image width updated from 120 to 60
z = 60 # image height updated from 120 to 60

In [24]:
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(classes, activation='softmax'))

model_2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [25]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [26]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size =Epoch 1/10
 10
67/67 [==============================] - 72s 1s/step - loss: 1.6135 - categorical_accuracy: 0.2150 - val_loss: 1.5221 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-1.61443-0.21719-1.52207-0.21000.h5
Epoch 2/10
67/67 [==============================] - 23s 338ms/step - loss: 1.5324 - categorical_accuracy: 0.3234 - val_loss: 1.5086 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-1.53244-0.32338-1.50864-0.28000.h5
Epoch 3/10
67/67 [==============================] - 24s 359ms/step - loss: 1.5815 - categorical_accuracy: 0.2637 - val_loss: 1.3803 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.58146-0.26368-1.38028-0.33000.h5
Epoch 4/10
67/67 [==============================] - 23s 34

`categorical_accuracy` = 50%  and `val_categorical_accuracy`= 54% . Deleting all low accuracy .h5 files.

### `Experiment 04`

**Changing Batch size to 20**

In [27]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [28]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train ; batch size = 20
Epoch 1/10
67/67 [==============================] - 84s 1s/step - loss: 0.8859 - categorical_accuracy: 0.6274 - val_loss: 1.1076 - val_categorical_accuracy: 0.5300

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-0.91984-0.61286-1.10763-0.53000.h5
Epoch 2/10
67/67 [==============================] - 28s 417ms/step - loss: 0.9143 - categorical_accuracy: 0.6318 - val_loss: 1.0809 - val_categorical_accuracy: 0.5600

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-0.91430-0.63184-1.08092-0.56000.h5
Epoch 3/10
67/67 [==============================] - 29s 436ms/step - loss: 0.9688 - categorical_accuracy: 0.5920 - val_loss: 0.9763 - val_categorical_accuracy: 0.5600

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-0.96880-0.59204-0.97630-0.56000.h5
Epoch 4/10
67/67 [==============================] - 29s 43

 Max `categorical_accuracy` at 80%  and `val_categorical_accuracy` at 58%

### `Experiment 05`

**Changing Batch size to 30**

In [29]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [30]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/10
67/67 [==============================] - 85s 1s/step - loss: 0.3214 - categorical_accuracy: 0.9069 - val_loss: 1.1441 - val_categorical_accuracy: 0.6250

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-0.31966-0.90063-1.14413-0.62500.h5
Epoch 2/10
67/67 [==============================] - 24s 355ms/step - loss: 0.3091 - categorical_accuracy: 0.8856 - val_loss: 1.2620 - val_categorical_accuracy: 0.5900

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-0.30906-0.88557-1.26195-0.59000.h5
Epoch 3/10
67/67 [==============================] - 24s 360ms/step - loss: 0.3236 - categorical_accuracy: 0.9005 - val_loss: 1.0628 - val_categorical_accuracy: 0.5600

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-0.32365-0.90050-1.06281-0.56000.h5
Epoch 4/10
67/67 [==============================] - 23s 33

Good accuracy with  `categorical_accuracy` at 90%  and `val_categorical_accuracy` at 63%

### `Experiment 06`

**Changing Batch size to 40**

In [31]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [32]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path = Epoch 1/10
 Project_data/train ; batch size = 40
67/67 [==============================] - 181s 3s/step - loss: 0.1841 - categorical_accuracy: 0.9411 - val_loss: 1.2335 - val_categorical_accuracy: 0.6167

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-0.18460-0.94145-1.23351-0.61667.h5
Epoch 2/10
67/67 [==============================] - 123s 2s/step - loss: 0.1745 - categorical_accuracy: 0.9496 - val_loss: 1.3138 - val_categorical_accuracy: 0.6000

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-0.17436-0.94936-1.31383-0.60000.h5
Epoch 3/10
67/67 [==============================] - 118s 2s/step - loss: 0.1738 - categorical_accuracy: 0.9429 - val_loss: 1.1321 - val_categorical_accuracy: 0.6300

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-0.17382-0.94293-1.13212-0.63000.h5
Epoch 4/10
67/67 [==============================] - 118s 2s/s

Max `categorical_accuracy` at 91%  and `val_categorical_accuracy` at 63%

### `Experiment 07`

**Changed Optimizer to Adadelta**

In [33]:
model_2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [34]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [35]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40Source path =  Project_data/train ; batch size = 40
Epoch 1/10

67/67 [==============================] - 183s 3s/step - loss: 0.2508 - categorical_accuracy: 0.9080 - val_loss: 1.7793 - val_categorical_accuracy: 0.6083

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-0.25298-0.90899-1.77930-0.60833.h5
Epoch 2/10
67/67 [==============================] - 120s 2s/step - loss: 0.1624 - categorical_accuracy: 0.9437 - val_loss: 1.9555 - val_categorical_accuracy: 0.6050

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-0.16580-0.94249-1.95551-0.60500.h5
Epoch 3/10
67/67 [==============================] - 119s 2s/step - loss: 0.0961 - categorical_accuracy: 0.9658 - val_loss: 2.5898 - val_categorical_accuracy: 0.5250

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-0.09605-0.96576-2.58981-0.52500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5

`Best Fit so Far ` `categorical_accuracy: 0.9947` - val_loss: 1.6844 - `val_categorical_accuracy: 0.6400`


Epoch 00010: saving model to model_init_2020-05-1918_24_07.458406/model-00010-0.01637-0.99473-1.68444-0.64000.h5

### `Experiment 08`

###  Changing `Epoch = 20`

In [36]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [37]:
import keras
model_2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [38]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [39]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - 185s 3s/step - loss: 0.0748 - categorical_accuracy: 0.9790 - val_loss: 2.3519 - val_categorical_accuracy: 0.5583

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-0.07103-0.97971-2.35195-0.55833.h5
Epoch 2/20
67/67 [==============================] - 122s 2s/step - loss: 0.0586 - categorical_accuracy: 0.9777 - val_loss: 2.2140 - val_categorical_accuracy: 0.5850

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-0.05885-0.97768-2.21399-0.58500.h5
Epoch 3/20
67/67 [==============================] - 122s 2s/step - loss: 0.0276 - categorical_accuracy: 0.9921 - val_loss: 2.4981 - val_categorical_accuracy: 0.6150

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-0.02760-0.99210-2.49806-0.61500.h5
Epoch 4/20
67/67 [==============================] - 121s 2s/s

`categorical_accuracy: 99.56%` - `val_categorical_accuracy: 67.00%`

### `Experiment 09`

# <a id='model_3'>Model_3</a>

In [40]:
model_3 = Sequential()
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_3.add(Activation('relu'))
model_3.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.5))
model_3.add(Dense(classes, activation='softmax'))
model_3.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [47]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 60 # image width
z = 60 # image height
num_epochs = 20 # increased from 10 to 20

In [48]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [49]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train Epoch 1/20
; batch size = 10
67/67 [==============================] - 71s 1s/step - loss: 2.1209 - categorical_accuracy: 0.2527 - val_loss: 3.5342 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-2.11318-0.25189-3.53425-0.25000.h5
Epoch 2/20
67/67 [==============================] - 22s 324ms/step - loss: 2.0141 - categorical_accuracy: 0.2985 - val_loss: 2.1869 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-2.01411-0.29851-2.18692-0.35000.h5
Epoch 3/20
67/67 [==============================] - 24s 360ms/step - loss: 2.0106 - categorical_accuracy: 0.2637 - val_loss: 3.0619 - val_categorical_accuracy: 0.2400

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-2.01059-0.26368-3.06192-0.24000.h5
Epoch 4/20
67/67 [==============================] - 23s 34

`For Model 3, with increase in epoch the accuracy dropped drastically below 40%`.

### `Experiment 10`

# <a id='model_4'>Model_4</a>

In [50]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]

In [51]:
# Define model
model_4 = Sequential()

model_4.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_4.add(Flatten())

model_4.add(Dense(nb_dense[0], activation='relu'))
model_4.add(Dropout(0.5))

model_4.add(Dense(nb_dense[1], activation='relu'))
model_4.add(Dropout(0.5))

#softmax layer
model_4.add(Dense(nb_dense[2], activation='softmax'))
model_4.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 60, 60, 8)     656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 60, 60, 8)     32        
_________________________________________________________________
activation_19 (Activation)   (None, 30, 60, 60, 8)     0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 15, 30, 30, 8)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 15, 30, 30, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 30, 30, 16)    64        
_________________________________________________________________
activation_20 (Activation)   (None, 15, 30, 30, 16)    0         
__________

In [52]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [53]:
model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 69s 1s/step - loss: 1.9899 - categorical_accuracy: 0.3218 - val_loss: 1.3533 - val_categorical_accuracy: 0.4500

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-1.99381-0.31825-1.35328-0.45000.h5
Epoch 2/10
67/67 [==============================] - 21s 316ms/step - loss: 1.7971 - categorical_accuracy: 0.2736 - val_loss: 1.1424 - val_categorical_accuracy: 0.5100

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-1.79713-0.27363-1.14240-0.51000.h5
Epoch 3/10
67/67 [==============================] - 22s 331ms/step - loss: 1.6367 - categorical_accuracy: 0.3383 - val_loss: 2.3671 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.63666-0.33831-2.36713-0.25000.h5
Epoch 4/10
67/67 [==============================] - 23s 33

`Accuracy Below 40%`

### `Experiment 11`

# <a id='model_5'>Model_5</a>

In [54]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1 # Keeping Channels to 1 from 3

In [55]:
model_5 = Sequential()
model_5.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_5.add(Activation('relu'))
model_5.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Flatten())
model_5.add(Dense(512, activation='relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.5))
model_5.add(Dense(classes, activation='softmax'))
model_5.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_23 (Conv3D)           (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_23 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_24 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

In [56]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20

In [57]:
model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 5
Source path =  Project_data/train ; batch size = 5
Epoch 1/20
67/67 [==============================] - 37s 554ms/step - loss: 2.4093 - categorical_accuracy: 0.1881 - val_loss: 5.2154 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-2.40934-0.18806-5.21541-0.22000.h5
Epoch 2/20
67/67 [==============================] - 32s 479ms/step - loss: 2.2058 - categorical_accuracy: 0.2169 - val_loss: 1.8742 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-2.19616-0.21752-1.87422-0.28000.h5
Epoch 3/20
67/67 [==============================] - 21s 307ms/step - loss: 2.2790 - categorical_accuracy: 0.2189 - val_loss: 5.8511 - val_categorical_accuracy: 0.2400

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-2.27901-0.21891-5.85111-0.24000.h5
Epoch 4/20
67/67 [==============================] - 22s 3

`Dropping the channels from 3 to 1 dropped the accuracy significantly below 40%.`

### `Experiment 12`

# <a id='model_6'>Model_6</a>

In [58]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

the below model is already run once, making no modifications, The cell no is irrelevant

In [59]:
model_6 = Sequential()

model_6.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_6.add(Flatten())

model_6.add(Dense(nb_dense[0], activation='relu'))
model_6.add(Dropout(0.5))

model_6.add(Dense(nb_dense[1], activation='relu'))
model_6.add(Dropout(0.5))

#softmax layer
model_6.add(Dense(nb_dense[2], activation='softmax'))
model_6.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_30 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [60]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30

In [61]:
model_6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - 78s 1s/step - loss: 6.5273 - categorical_accuracy: 0.2587 - val_loss: 7.0941 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-6.57813-0.25792-7.09410-0.23000.h5
Epoch 2/30
67/67 [==============================] - 21s 311ms/step - loss: 2.4784 - categorical_accuracy: 0.3234 - val_loss: 1.7358 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-2.47840-0.32338-1.73577-0.20000.h5
Epoch 3/30
67/67 [==============================] - 25s 380ms/step - loss: 1.8185 - categorical_accuracy: 0.2886 - val_loss: 1.3356 - val_categorical_accuracy: 0.3800

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.81851-0.28856-1.33561-0.38000.h5
Epoch 4/30
67/67 [==============================] - 25s 37

67/67 [==============================] - 24s 364ms/step - loss: 0.7477 - categorical_accuracy: 0.7164 - val_loss: 0.6601 - val_categorical_accuracy: 0.7300

Epoch 00029: saving model to model_init_2020-05-2205_36_02.814533/model-00029-0.74773-0.71642-0.66014-0.73000.h5
Epoch 30/30
67/67 [==============================] - 25s 380ms/step - loss: 0.8837 - categorical_accuracy: 0.6716 - val_loss: 0.6407 - val_categorical_accuracy: 0.7500

Epoch 00030: saving model to model_init_2020-05-2205_36_02.814533/model-00030-0.88373-0.67164-0.64071-0.75000.h5


`Model 6 : Stable Model: categorical_accuracy: 76%` `val_categorical_accuracy: 072%`

### `Experiment 13`

# <a id='model_7'>Model_7</a>

In [62]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

In [63]:
# Define model
model_7 = Sequential()

model_7.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

model_7.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

model_7.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

model_7.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_7.add(Flatten())

model_7.add(Dense(nb_dense[0], activation='relu'))
model_7.add(Dropout(0.5))

model_7.add(Dense(nb_dense[1], activation='relu'))
model_7.add(Dropout(0.5))

#softmax layer
model_7.add(Dense(nb_dense[2], activation='softmax'))
model_7.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_7.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_33 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_33 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_23 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_34 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_34 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [64]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [65]:
model_7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 82s 1s/step - loss: 5.0021 - categorical_accuracy: 0.3109 - val_loss: 1.2963 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-5.02211-0.31071-1.29626-0.41000.h5
Epoch 2/10
67/67 [==============================] - 24s 365ms/step - loss: 2.4799 - categorical_accuracy: 0.2537 - val_loss: 1.4830 - val_categorical_accuracy: 0.3400

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-2.47993-0.25373-1.48296-0.34000.h5
Epoch 3/10
67/67 [==============================] - 27s 396ms/step - loss: 1.7884 - categorical_accuracy: 0.3532 - val_loss: 1.6908 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.78838-0.35323-1.69079-0.29000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 

`Model 7: Poor accuracy, will not consider`

### `Experiment 14`

# <a id='model_8'>Model_8</a>

In [66]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

In [67]:
# Define model
model_8 = Sequential()

model_8.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

model_8.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

model_8.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

model_8.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_8.add(Flatten())

model_8.add(Dense(nb_dense[0], activation='relu'))
model_8.add(Dropout(0.5))

model_8.add(Dense(nb_dense[1], activation='relu'))
model_8.add(Dropout(0.5))

#softmax layer
model_8.add(Dense(nb_dense[2], activation='softmax'))
model_8.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_8.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_37 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_27 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_38 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [68]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [69]:
model_8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/valSource path =   ; batch size = 10
Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 80s 1s/step - loss: 5.4089 - categorical_accuracy: 0.2512 - val_loss: 3.1684 - val_categorical_accuracy: 0.1900

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-5.45291-0.25038-3.16838-0.19000.h5
Epoch 2/10
67/67 [==============================] - 26s 385ms/step - loss: 2.3922 - categorical_accuracy: 0.2985 - val_loss: 1.9809 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-2.39221-0.29851-1.98086-0.30000.h5
Epoch 3/10
67/67 [==============================] - 26s 381ms/step - loss: 1.7383 - categorical_accuracy: 0.3383 - val_loss: 1.5260 - val_categorical_accuracy: 0.3500

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.73826-0.33831-1.52595-0.35000.h5
Epoch 4/10
67/67 [==============================] - 26s 38

`Model 8 - Poor accuracy`

### `Experiment 15`

# <a id='model_9'>Model_9</a>

In [70]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

In [71]:
# Define model
model_9 = Sequential()

model_9.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

model_9.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

model_9.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

model_9.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_9.add(Activation('relu'))
model_9.add(Dropout(0.25))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_9.add(Flatten())

model_9.add(Dense(nb_dense[0], activation='relu'))
model_9.add(Dropout(0.5))

model_9.add(Dense(nb_dense[1], activation='relu'))
model_9.add(Dropout(0.5))

#softmax layer
model_9.add(Dense(nb_dense[2], activation='softmax'))
model_9.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_9.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_41 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_23 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_41 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_31 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_42 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_24 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_42 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [72]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [73]:
model_9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = Epoch 1/10
10
67/67 [==============================] - 85s 1s/step - loss: 2.1915 - categorical_accuracy: 0.3050 - val_loss: 1.3406 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-2.20270-0.30468-1.34058-0.38000.h5
Epoch 2/10
67/67 [==============================] - 25s 370ms/step - loss: 1.6011 - categorical_accuracy: 0.3433 - val_loss: 1.6974 - val_categorical_accuracy: 0.1900

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-1.60109-0.34328-1.69737-0.19000.h5
Epoch 3/10
67/67 [==============================] - 27s 396ms/step - loss: 1.6023 - categorical_accuracy: 0.3383 - val_loss: 1.3454 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.60233-0.33831-1.34543-0.44000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 

Epoch 10/10 `categorical_accuracy: 0.6866` - `val_categorical_accuracy: 0.6900`

Epoch 00010: saving model to model_init_2020-05-1918_24_07.458406/model-00010-0.81188-0.68657-0.80880-0.69000.h5

### `Experiment 16`

# <a id='model_10'>Model_10</a> 

In [74]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

In [75]:
# Define model
model_10 = Sequential()

model_10.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_10.add(BatchNormalization())
model_10.add(Activation('relu'))

model_10.add(MaxPooling3D(pool_size=(2,2,2)))

model_10.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_10.add(BatchNormalization())
model_10.add(Activation('relu'))

model_10.add(MaxPooling3D(pool_size=(2,2,2)))

model_10.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_10.add(BatchNormalization())
model_10.add(Activation('relu'))

model_10.add(MaxPooling3D(pool_size=(2,2,2)))

model_10.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_10.add(Activation('relu'))
model_10.add(Dropout(0.25))

model_10.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_10.add(Flatten())

model_10.add(Dense(nb_dense[0], activation='relu'))
model_10.add(Dropout(0.5))

model_10.add(Dense(nb_dense[1], activation='relu'))
model_10.add(Dropout(0.5))

#softmax layer
model_10.add(Dense(nb_dense[2], activation='softmax'))

In [76]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_10.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_10.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_45 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_26 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_45 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_35 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_46 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_27 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_46 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [77]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [78]:
batch_size = 10
num_epochs = 20
model_10.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 84s 1s/step - loss: 1.8811 - categorical_accuracy: 0.3124 - val_loss: 1.3335 - val_categorical_accuracy: 0.4500

Epoch 00001: saving model to model_init_2020-05-2205_36_02.814533/model-00001-1.88566-0.31222-1.33353-0.45000.h5
Epoch 2/20
67/67 [==============================] - 25s 370ms/step - loss: 1.4650 - categorical_accuracy: 0.3831 - val_loss: 1.4067 - val_categorical_accuracy: 0.4300

Epoch 00002: saving model to model_init_2020-05-2205_36_02.814533/model-00002-1.46500-0.38308-1.40666-0.43000.h5
Epoch 3/20
67/67 [==============================] - 27s 401ms/step - loss: 1.5077 - categorical_accuracy: 0.3234 - val_loss: 1.2749 - val_categorical_accuracy: 0.4200

Epoch 00003: saving model to model_init_2020-05-2205_36_02.814533/model-00003-1.50773-0.32338-1.27489-0.42000.h5
Epoch 4/20
67/67 [==============================] - 27s 40

`Categorical_accuracy = 58%, Val_categorical_accuracy = 62%, Observation - Accuracy Decreased`

#### `After experineting with different weights ,We will go  with Expermint 8 as our final model`

`Reading the hdf5.f5 file`

In [28]:
import numpy as np
import h5py 

In [29]:
filepath = 'model_init_2020-05-2205_36_02.814533/Experiment_08_model-00012-0.01036-0.99561-2.59650-0.67000.h5'
filepath = 'model_init_2020-05-2205_36_02.814533/Experiment_01_model-00010-1.32115-0.44279-1.11734-0.58000.h5'

In [30]:
with h5py.File(filepath, 'r') as hdf:
    ls = list(hdf.keys())
    print('list of data sets in this file: \n ', ls) # ['model_weights', 'optimizer_weights']
    
    data_model_weights = hdf.get('model_weights')
    model_weights = np.array(data_model_weights)

    
    data_optimizer_weights = hdf.get('optimizer_weights')
    optimizer_weights = np.array(data_optimizer_weights)
    
    

list of data sets in this file: 
  ['model_weights', 'optimizer_weights']


In [31]:
optimizer_weights

array(['Adam', 'training'], dtype='<U8')

In [32]:
model_weights

array(['activation_1', 'activation_2', 'activation_3', 'activation_4',
       'batch_normalization_1', 'batch_normalization_2',
       'batch_normalization_3', 'batch_normalization_4', 'conv3d_1',
       'conv3d_2', 'conv3d_3', 'conv3d_4', 'dense_1', 'dense_2',
       'dense_3', 'dropout_1', 'dropout_2', 'flatten_1',
       'max_pooling3d_1', 'max_pooling3d_2', 'max_pooling3d_3',
       'max_pooling3d_4'], dtype='<U21')

<br><br><br><br><br><br>

`End - Observation is present in the Gesture_Recognition_Observation.pdf` 